In [1]:
# imports
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset
from datasets import Features
from datasets import Sequence
from datasets import ClassLabel
from datasets import Value
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertConfig, AutoModelForSequenceClassification, \
    DataCollatorForTokenClassification, TrainingArguments, Trainer
import torch

In [2]:
from huggingface_hub import notebook_login

#notebook_login()

In [3]:
RES_PATH = os.path.abspath("../resources/data/") + "/"

In [4]:
def load_dataset_dict(dataset_name, dataset_path, limit=None):
    train_file = dataset_path + dataset_name + "/train.json"
    test_file = dataset_path + dataset_name + "/test.json"
    dev_file = dataset_path + dataset_name + "/dev.json"

    return load_dataset("json", data_files={"train":train_file, "validation":dev_file, "test":test_file},
                        features=Features({
                            "id": Value(dtype="string", id=None),
                            "label": Sequence(ClassLabel(num_classes=3, names=["B", "I", "O"]), length=-1, id=None),
                            "sentence": Sequence(Value(dtype="string", id=None), length=-1, id=None),
                        })).rename_column("label", "labels")

In [5]:
rest_dataset_dict = load_dataset_dict("test", RES_PATH)
#rest_dataset_dict = load_dataset_dict("rest", RES_PATH)

In [6]:
rest_dataset_dict["train"][0]

{'id': '0',
 'labels': [0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2],
 'sentence': ['Keyboard',
  'is',
  'great',
  'but',
  'primary',
  'and',
  'secondary',
  'control',
  'buttons',
  'could',
  'be',
  'more',
  'durable',
  '.']}

In [7]:
#label_list = rest_dataset_dict["train"].features["label"].feature.names # ["B", "I", "O"]
label_list = ["B", "I", "O"]
label_list

['B', 'I', 'O']

In [8]:
bert_auto_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
example = rest_dataset_dict["train"][0]
tokenized_input_rest = bert_auto_tokenizer(example["sentence"], is_split_into_words=True, padding=True, truncation=True)
tokens_rest = bert_auto_tokenizer.convert_ids_to_tokens(tokenized_input_rest["input_ids"])
tokens_rest

['[CLS]',
 'keyboard',
 'is',
 'great',
 'but',
 'primary',
 'and',
 'secondary',
 'control',
 'buttons',
 'could',
 'be',
 'more',
 'durable',
 '.',
 '[SEP]']

In [10]:
def tokenize_and_align_labels(batch_data):
    tokenized_inputs = bert_auto_tokenizer(batch_data["sentence"], is_split_into_words=True, padding=True, truncation=True)

    labels = []
    for i, label in enumerate(batch_data["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100 -> ignored by PT
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_rest_dataset_dict = rest_dataset_dict.map(tokenize_and_align_labels, batched=True)

In [12]:
tokenized_rest_dataset_dict["train"].features

{'id': Value(dtype='string', id=None),
 'labels': Sequence(feature=ClassLabel(names=['B', 'I', 'O'], id=None), length=-1, id=None),
 'sentence': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=bert_auto_tokenizer, padding=True)

In [14]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [15]:
labels = [label_list[i] for i in example["labels"]]

In [16]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    for prediction, label in zip(predictions, labels):
        for (p, l) in zip(prediction, label):
            if l != -100:
                true_predictions.append(label_list[p])

    true_labels = []
    for prediction, label in zip(predictions, labels):
        for (p, l) in zip(prediction, label):
            if l != -100:
                true_labels.append(label_list[l])

    true_labels = [label2id[item] for item in true_labels]
    true_predictions = [label2id[item] for item in true_predictions]

    results = {}
    results.update(accuracy_metric.compute(predictions=true_predictions, references=true_labels))
    results.update(precision_metric.compute(predictions=true_predictions, references=true_labels, average="micro"))
    results.update(recall_metric.compute(predictions=true_predictions, references=true_labels, average="micro"))
    results.update(f1_metric.compute(predictions=true_predictions, references=true_labels, average="micro"))

    return results

In [17]:
id2label = {
    0: "B",
    1: "I",
    2: "O",
}
label2id = {
    "B": 0,
    "I": 1,
    "O": 2
}

In [18]:
bert_token_classificator = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="aspect_extraction_restaurant_reviews",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    #save_steps=100.0,
    #save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True)

In [20]:
trainer = Trainer(
    model=bert_token_classificator,
    args=training_args,
    train_dataset=tokenized_rest_dataset_dict["train"],
    eval_dataset=tokenized_rest_dataset_dict["test"],
    tokenizer=bert_auto_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/jannik/DataspellProjects/nlp-project-ae/notebooks/aspect_extraction_restaurant_reviews is already a clone of https://huggingface.co/jannikseus/aspect_extraction_restaurant_reviews. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [21]:
trainer.train()

/Users/jannik/DataspellProjects/nlp-project-ae/venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=2, training_loss=1.1182729005813599, metrics={'train_runtime': 24.778, 'train_samples_per_second': 1.211, 'train_steps_per_second': 0.081, 'total_flos': 283256959500.0, 'train_loss': 1.1182729005813599, 'epoch': 1.0})

In [22]:
trainer.push_to_hub()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

To https://huggingface.co/jannikseus/aspect_extraction_restaurant_reviews
   54d5b4a..1cda4cb  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
trainer.evaluate()

{'eval_loss': 0.9864752292633057,
 'eval_accuracy': 0.5597014925373134,
 'eval_precision': 0.5597014925373134,
 'eval_recall': 0.5597014925373134,
 'eval_f1': 0.5597014925373134,
 'eval_runtime': 0.5613,
 'eval_samples_per_second': 14.253,
 'eval_steps_per_second': 1.782,
 'epoch': 1.0}

In [25]:
trainer.predict(tokenized_rest_dataset_dict["validation"])

PredictionOutput(predictions=array([[[ 2.60803550e-01, -1.20568916e-01,  1.01086825e-01],
        [ 6.14641048e-02, -6.55389205e-02,  4.87749055e-02],
        [ 2.67148077e-01, -6.06393814e-03,  2.71650434e-01],
        [ 6.88832104e-02, -2.08010897e-01,  2.77577132e-01],
        [ 1.65450573e-01, -3.58482659e-01,  2.26256847e-01],
        [ 7.63909519e-02, -2.54156619e-01,  1.70909703e-01],
        [ 3.29094902e-02, -1.20776355e-01,  3.96264732e-01],
        [ 3.01099688e-01, -1.11149773e-01,  3.32548320e-01],
        [ 2.89300323e-01, -1.34115368e-01,  2.61822402e-01],
        [ 2.22279117e-01, -1.50743514e-01,  1.23772010e-01],
        [ 1.51342154e-01, -3.14011812e-01,  2.73642033e-01],
        [-7.39142299e-04, -4.92152758e-02,  9.84793156e-02],
        [-3.85688134e-02,  6.42010868e-02,  1.10937528e-01],
        [ 1.64174020e-01,  1.68363675e-01, -2.67640874e-02],
        [ 1.95947886e-01,  2.27761894e-01,  1.84585929e-01],
        [ 4.50445950e-01, -6.62415773e-02,  2.88740218e-